In [1]:
# Import libraries
import pandas as pd
import numpy as np
import random
import math

In [2]:
# Declare variables
# iterations (n)
n = 5000

# winning odds (r), casino rate is 0.44
r = 0.49

# base bet (b)
b = 100

# table maximum (M)
m = 1000

In [3]:
# Calculate max consecutive losses before table max is hit (mcl)

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

mcl = round_down(math.log(m/b,2))+2
print(mcl)

5.0


In [4]:
# Generate win/loss array
win_loss_array = []
for i in range (0,n):
    win_loss_generation = random.choices([0,1], weights = [1-r,r])
    win_loss_number = win_loss_generation[0]
    if win_loss_number >= 0.5:
        win_loss_array += 'W'
    else:
        win_loss_array += 'L'

# Generate dummy array for the simulated data
consecutive_l = [None] * n
bet_amount = [None] * n
round_cashflow = [None] * n
max_bet_counter = [None] * n

# Print first 10 items of the resultant arrays
print('The first 10 items of the win/loss array:')
print(win_loss_array[:10])

The first 10 items of the win/loss array:
['L', 'W', 'W', 'L', 'W', 'L', 'W', 'W', 'L', 'L']


In [5]:
# Generate base dataframe
df = pd.DataFrame({'W/L' : win_loss_array[:],
                   'Consecutive L': consecutive_l[:],
                   'Max Bet Counter': max_bet_counter[:],
                   'Bet Amount': bet_amount[:],
                   'Round Cashflow': round_cashflow[:],
                  })
print('The first 5 items of the resultant dataframe:')
display(df.head())

The first 5 items of the resultant dataframe:


,W/L,Consecutive L,Max Bet Counter,Bet Amount,Round Cashflow
0,L,None,None,None,None
1,W,None,None,None,None
2,W,None,None,None,None
3,L,None,None,None,None
4,W,None,None,None,None


In [6]:
# Count consecutive L
if df.iloc[0,0] == 'W':
    df.iloc[0,1] = 0
else:
    df.iloc[0,1] = 1
    
for i in range(1,n):
    if df.iloc[i,0] == 'L':
        df.iloc[i,1] = df.iloc[i-1,1]+1
    else:
        df.iloc[i,1] = 0

In [7]:
# Generate max bet counter
df.iloc[0,2] = 0

for i in range(1,n):
    # Scenario 1: max bet counter is 0 and max consecutive loss is not reached
    if df.iloc[i,1] < mcl and df.iloc[i-1,2] == 0:
        df.iloc[i,2] = 0
    # Scenario 2: max bet counter is 0 and we lost the first max table bet
    elif df.iloc[i,1] == mcl and df.iloc[i-1,2] == 0 and df.iloc[i,0] == 'L':
        df.iloc[i,2] = 2
    # Scenario 3: max bet counter has started counting and we l lose
    elif df.iloc[i-1,2] > 0 and df.iloc[i,0] == 'L':
        df.iloc[i,2] = df.iloc[i-1,2]+1
    # Scenario 4: max bet counter has started counting and we win
    elif  df.iloc[i-1,2] > 0 and df.iloc[i,0] == 'W':
        df.iloc[i,2] = df.iloc[i-1,2]-1

In [8]:
# Generate bet amount

# Start initial bet (b)
df.iloc[0,3] = b

# Second bet is always (b) regardless of the outcome of the first round
df.iloc[1,3] = b

for i in range(2,n):
    # Scenario 1: max bet not hit, last 2 games were losses
    if df.iloc[i,2] == 0 and df.iloc[i-1,0] == 'L' and df.iloc[i-2,0] == 'L':
        df.iloc[i,3] = df.iloc[i-1,3]*2
    # Scenario 2: max bet not hit, won last game
    elif df.iloc[i,2] == 0 and df.iloc[i-1,0] == 'W':
        df.iloc[i,3] = b
    # Scenario 3: max bet not hit, previous round lost but the one before is win
    elif df.iloc[i,2] == 0 and df.iloc[i-1,0] == 'L' and df.iloc[i-2,0] == 'W':
        df.iloc[i,3] = b
    # Scenario 4: max bet hit, need to place max bet
    elif df.iloc[i,2] > 0:
        df.iloc[i,3] = (2**(mcl-2))*10

In [9]:
# Generate round cashflow
for i in range(0,n):
    if df.iloc[i,0] == 'L':
        df.iloc[i,4] = df.iloc[i,3]*-1
    else:
        df.iloc[i,4] = df.iloc[i,3]

In [10]:
# Generate total cashflow of previous rounds
df['Total Cashflow'] = df['Round Cashflow'].cumsum()

In [11]:
# check last 5 rows to check for simulation completion
df.iloc[-3:]

,W/L,Consecutive L,Max Bet Counter,Bet Amount,Round Cashflow,Total Cashflow
4997,W,0,140,80.0,80.0,-11320.0
4998,L,1,141,80.0,-80.0,-11400.0
4999,L,2,142,80.0,-80.0,-11480.0


In [12]:
# Summary statistics
print('Summary statistics: \n')

print("Total iterations ran: %s" % '{:,}'.format((n)))
print("Total wins: %s" % '{:,}'.format((df['W/L'].value_counts()['W'])))
print("Total losses: %s" % '{:,}'.format((df['W/L'].value_counts()['L'])))
print("Win rate: %s" % "{0:.02%}".format((df['W/L'].value_counts()['W'])/n))
print("Expected Win rate: %s \n" % "{0:.02%}".format(r))

print("Most consecutive losses in a row: %s" % (int(math.log(abs(df['Round Cashflow'].min()),b))+2))
print("Lowest Round Cashflow: %s" % '{:,}'.format(df['Round Cashflow'].min()))
print("Lowest Total Cashflow: %s" % '{:,}'.format(df['Total Cashflow'].min()))
print("Final Total Cashflow: %s" % '{:,}'.format(df.iloc[-1,5]))

Summary statistics: 

Total iterations ran: 5,000
Total wins: 2,428
Total losses: 2,572
Win rate: 48.56%
Expected Win rate: 49.00% 

Most consecutive losses in a row: 3
Lowest Round Cashflow: -400
Lowest Total Cashflow: -12,120.0
Final Total Cashflow: -11,480.0


In [13]:
# Generate full dataframe for sanity check
df.head(50)

,W/L,Consecutive L,Max Bet Counter,Bet Amount,Round Cashflow,Total Cashflow
0,L,1,0,100,-100,-100
1,W,0,0,100,100,0
2,W,0,0,100,100,100
3,L,1,0,100,-100,0
4,W,0,0,100,100,100
5,L,1,0,100,-100,0
6,W,0,0,100,100,100
7,W,0,0,100,100,200
8,L,1,0,100,-100,100
9,L,2,0,100,-100,0
